IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\FAULT_M3_5


In [3]:
guasto=1

VOLO m3 FAULT 10%

In [4]:
rcou_m3_fault5 = pd.read_csv("RCOU.csv")
rcou_m3_fault5 = rcou_m3_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m3_fault5 = rcou_m3_fault5[((rcou_m3_fault5['C9'] >= 1350) & (rcou_m3_fault5['C10'] >= 1350) & (rcou_m3_fault5['C11'] >= 1350) & (rcou_m3_fault5['C12'] >= 1350) & (rcou_m3_fault5['C13'] >= 1350) & (rcou_m3_fault5['C14']>= 1350))]
rcou_m3_fault5=rcou_m3_fault5.reset_index(drop=True)
display(rcou_m3_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,204202091,1406,1380,1413,1372,1424,1359
1,204212099,1369,1459,1408,1424,1358,1468
2,204222805,1422,1454,1486,1386,1470,1405
3,204233384,1471,1448,1448,1471,1457,1462
4,204243252,1509,1450,1435,1522,1463,1497
...,...,...,...,...,...,...,...
8582,303987906,1536,1428,1542,1421,1443,1524
8583,303999574,1533,1428,1529,1433,1427,1534
8584,304010535,1532,1424,1521,1437,1419,1537
8585,304021075,1523,1428,1498,1456,1385,1557


In [5]:
min=rcou_m3_fault5['TimeUS'][0]
max=rcou_m3_fault5['TimeUS'][len(rcou_m3_fault5)-1]
print(max)
print(min)

304031184
204202091


In [6]:
xkf1_m3_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault5 = xkf1_m3_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault5 = xkf1_m3_fault5[((xkf1_m3_fault5['TimeUS'] >= min) & (xkf1_m3_fault5['TimeUS'] <= max	))]
xkf1_m3_fault5 = xkf1_m3_fault5.reset_index(drop=True)
print(xkf1_m3_fault5)

         TimeUS  Roll  Pitch   Yaw
0     204210668  0.88  -0.77  2.52
1     204220693  0.85  -0.83  2.51
2     204230930  0.91  -0.79  2.53
3     204241781  0.94  -0.72  2.56
4     204251811  1.02  -0.76  2.56
...         ...   ...    ...   ...
9085  303985816  1.10  -0.30  2.01
9086  303997185  1.06  -0.25  2.00
9087  304007782  0.97  -0.22  1.97
9088  304018840  0.93  -0.14  1.94
9089  304029034  0.96  -0.06  1.90

[9090 rows x 4 columns]


In [7]:
att_m3_fault5 = pd.read_csv("ATT.csv")
att_m3_fault5 = att_m3_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault5 = att_m3_fault5[((att_m3_fault5['TimeUS'] >= min) & (att_m3_fault5['TimeUS'] <= max	))]
att_m3_fault5=att_m3_fault5.reset_index(drop=True)
print(att_m3_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0     204211309     0.00  0.88      0.00  -0.77    2.49  2.52
1     204222059     0.00  0.85      0.00  -0.83    2.49  2.51
2     204232678     0.00  0.91      0.00  -0.79    2.49  2.53
3     204242578     0.00  0.94      0.00  -0.72    2.49  2.56
4     204253192     0.00  1.02      0.00  -0.76    2.49  2.56
...         ...      ...   ...       ...    ...     ...   ...
9086  303987111     2.69  1.10      1.47  -0.30    1.71  2.01
9087  303998743     2.66  1.06      1.49  -0.25    1.71  2.00
9088  304009828     2.65  0.97      1.52  -0.22    1.71  1.97
9089  304020291     2.64  0.93      1.55  -0.14    1.71  1.94
9090  304030671     2.63  0.96      1.58  -0.06    1.71  1.90

[9091 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_fault5)>len(xkf1_m3_fault5):
    to_add=att_m3_fault5[len(xkf1_m3_fault5):]
    att_m3_fault5=att_m3_fault5[:len(xkf1_m3_fault5)]

if len(xkf1_m3_fault5)>len(att_m3_fault5):
    to_add=xkf1_m3_fault5[len(att_m3_fault5):]

for idx,i in enumerate(att_m3_fault5['Roll']):
    if(xkf1_m3_fault5['Roll'][idx] != i):
        att_m3_fault5['Roll'][idx] = ((att_m3_fault5['Roll'][idx] + xkf1_m3_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Pitch']):
    if(xkf1_m3_fault5['Pitch'][idx] != i):
        att_m3_fault5['Pitch'][idx] = ((att_m3_fault5['Pitch'][idx] + xkf1_m3_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Yaw']):
    if(xkf1_m3_fault5['Yaw'][idx] != i):
        att_m3_fault5['Yaw'][idx] = ((att_m3_fault5['Yaw'][idx] + xkf1_m3_fault5['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_fault5=pd.concat([att_m3_fault5,to_add])

print(att_m3_fault5)

C:\Users\mgale\AppData\Local\Temp\ipykernel_17676\3102436011.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_fault5['Roll'][idx] = ((att_m3_fault5['Roll'][idx] + xkf1_m3_fault5['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_17676\3102436011.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_fault5['Pitch'][idx] = ((att_m3_fault5['Pitch'][idx] + xkf1_m3_fault5['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_17676\3102436011.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0     204211309     0.00  0.880      0.00 -0.770    2.49  2.520
1     204222059     0.00  0.850      0.00 -0.830    2.49  2.510
2     204232678     0.00  0.910      0.00 -0.790    2.49  2.530
3     204242578     0.00  0.940      0.00 -0.720    2.49  2.560
4     204253192     0.00  1.020      0.00 -0.760    2.49  2.560
...         ...      ...    ...       ...    ...     ...    ...
9086  303987111     2.69  1.080      1.47 -0.275    1.71  2.005
9087  303998743     2.66  1.015      1.49 -0.235    1.71  1.985
9088  304009828     2.65  0.950      1.52 -0.180    1.71  1.955
9089  304020291     2.64  0.945      1.55 -0.100    1.71  1.920
9090  304030671     2.63  0.960      1.58 -0.060    1.71  1.900

[9091 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault5 = esc_0_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_fault5 = esc_0_m3_fault5[((esc_0_m3_fault5['TimeUS'] >= min) & (esc_0_m3_fault5['TimeUS'] <= max))]
esc_0_m3_fault5=esc_0_m3_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m3_fault5 = esc_1_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_fault5 = esc_1_m3_fault5[((esc_1_m3_fault5['TimeUS'] >= min) & (esc_1_m3_fault5['TimeUS'] <= max))]
esc_1_m3_fault5=esc_1_m3_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m3_fault5 = esc_2_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_fault5 = esc_2_m3_fault5[((esc_2_m3_fault5['TimeUS'] >= min) & (esc_2_m3_fault5['TimeUS'] <= max))]
esc_2_m3_fault5=esc_2_m3_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m3_fault5 = esc_3_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_fault5 = esc_3_m3_fault5[((esc_3_m3_fault5['TimeUS'] >= min) & (esc_3_m3_fault5['TimeUS'] <= max))]
esc_3_m3_fault5=esc_3_m3_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m3_fault5 = esc_4_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_fault5 = esc_4_m3_fault5[((esc_4_m3_fault5['TimeUS'] >= min) & (esc_4_m3_fault5['TimeUS'] <= max))]
esc_4_m3_fault5=esc_4_m3_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m3_fault5 = esc_5_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_fault5 = esc_5_m3_fault5[((esc_5_m3_fault5['TimeUS'] >= min) & (esc_5_m3_fault5['TimeUS'] <= max))]
esc_5_m3_fault5=esc_5_m3_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault5 = imu_0_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault5 = imu_0_m3_fault5[((imu_0_m3_fault5['TimeUS'] >= min) & (imu_0_m3_fault5['TimeUS'] <= max))]

imu_1_m3_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault5 = imu_1_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault5 = imu_1_m3_fault5[((imu_1_m3_fault5['TimeUS'] >= min) & (imu_1_m3_fault5['TimeUS'] <= max))]

imu_2_m3_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault5 = imu_2_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault5 = imu_2_m3_fault5[((imu_2_m3_fault5['TimeUS'] >= min) & (imu_2_m3_fault5['TimeUS'] <= max))]

imu_m3_fault5 = pd.concat((imu_0_m3_fault5, imu_1_m3_fault5, imu_2_m3_fault5))
imu_m3_fault5=imu_m3_fault5.groupby(imu_m3_fault5.TimeUS, as_index=False).mean()

display(imu_m3_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,204204632,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536
1,204206099,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001
2,204209242,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777
3,204211153,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143
4,204213901,-0.035754,0.064878,0.045289,-0.319344,0.122338,-10.160297
...,...,...,...,...,...,...,...
34503,304020133,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323
34504,304022922,0.050244,0.063406,-0.067958,0.036619,0.395237,-10.919890
34505,304025102,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300
34506,304027773,-0.008855,0.096466,-0.051342,-0.315915,0.138011,-10.782627


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_fault5 = pd.merge_ordered(imu_m3_fault5,att_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_0_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_1_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_2_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_3_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_4_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_5_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")
m3_fault5 = pd.merge_ordered(m3_fault5, rcou_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m3_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

9091
9945
10796
11653
12499
13337
14183


In [13]:
m3_fault5 = m3_fault5[~m3_fault5.TimeUS.isin(index_to_remove)]
m3_fault5["TimeUS"] = m3_fault5["TimeUS"] - m3_fault5.iloc[0]["TimeUS"]
m3_fault5["TimeUS"] = m3_fault5["TimeUS"].astype(int)
m3_fault5["Guasto"] = guasto

In [14]:
from datetime import timedelta
m3_fault5=m3_fault5.reset_index(drop=True)
m3_fault5['TimeUS'] = pd.to_datetime(m3_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1
1,00:00:00.001467,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1
2,00:00:00.004610,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1
3,00:00:00.006521,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143,0.00,0.880,0.00,...,2.74,1781.0,3.11,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1
4,00:00:00.009269,-0.035754,0.064878,0.045289,-0.319344,0.122338,-10.160297,0.00,0.880,0.00,...,2.74,1781.0,3.11,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34503,00:01:39.815501,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323,2.65,0.950,1.52,...,1.96,5735.0,3.18,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1
34504,00:01:39.818290,0.050244,0.063406,-0.067958,0.036619,0.395237,-10.919890,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1
34505,00:01:39.820470,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1
34506,00:01:39.823141,-0.008855,0.096466,-0.051342,-0.315915,0.138011,-10.782627,2.64,0.945,1.55,...,1.96,5735.0,3.18,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})','guasto'])
    for i in range(int(len(m3_fault5)/350)):
        V1=m3_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2,'guasto':guasto}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_17676\3002095869.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_17676\3002095869.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),guasto_x,...,guasto_y,<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),guasto
0,0.0,0.013701,0.008087,-0.480919,0.090836,49.0,61.0,2.189368,2.044501,1.0,...,1.0,4.112422,0.932041,-1.608298,4.223908,123.0,120.0,40.464358,22.418345,1.0
1,1.0,0.013688,0.008064,-0.473563,0.090708,64.0,58.0,2.568676,2.307767,1.0,...,1.0,4.110710,0.930417,-1.605529,4.222051,128.0,125.0,38.973694,21.743146,1.0
2,2.0,0.013755,0.008042,-0.468247,0.090602,68.0,67.0,2.693643,2.263490,1.0,...,1.0,4.109008,0.928796,-1.602728,4.220203,131.0,128.0,37.469860,20.843054,1.0
3,3.0,0.013779,0.008020,-0.461473,0.090482,68.0,69.0,2.429212,2.100722,1.0,...,1.0,4.107316,0.927178,-1.599894,4.218365,128.0,125.0,35.957402,19.727780,1.0
4,4.0,0.013733,0.007998,-0.454139,0.090354,69.0,60.0,2.344589,2.099998,1.0,...,1.0,4.105634,0.925564,-1.597029,4.216537,131.0,128.0,34.440743,18.410693,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,93.0,0.003700,0.007169,-0.161793,0.084654,0.0,113.0,6.039256,3.402165,1.0,...,1.0,3.990113,0.793431,-1.248281,4.088110,-159.0,-158.0,50.799356,30.471235,1.0
94,94.0,0.003561,0.007161,-0.158258,0.084604,0.0,96.0,6.062621,3.458822,1.0,...,1.0,3.988944,0.792252,-1.243809,4.086826,-171.0,-170.0,51.965549,30.995158,1.0
95,95.0,0.003695,0.007153,-0.160980,0.084563,0.0,113.0,6.207388,3.538484,1.0,...,1.0,3.987780,0.791076,-1.239324,4.085547,169.0,170.0,53.075969,31.501536,1.0
96,96.0,0.003690,0.007137,-0.154470,0.084468,0.0,112.0,6.289504,3.595596,1.0,...,1.0,3.986622,0.789902,-1.234825,4.084273,-173.0,-172.0,54.128245,31.989306,1.0


In [18]:
path_file = path_file.replace(r"M3\FAULT_M3_5", "")
os.chdir(path_file)
m3_fault5.to_csv('m3_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
